In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import json
from aiida import load_profile
from aiida.orm import StructureData, load_code, Float, load_node
from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData
from aiida.engine import submit
from ase.atoms import Atoms
from ase.visualize import view
from aiida.plugins import DataFactory 
from pathlib import Path
from aiida_quantumespresso.common.types import SpinType
from aiida.orm import Int
from copy import deepcopy
import os

load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

In [2]:

# ! Currently, for whatever reason, the codes are duplicated
# pw_code = load_code('qe-dev-pw@lumi-small')
# hp_code = load_code('qe-dev-hp@lumi-small')
pw_code = load_code(2182)
hp_code = load_code(2183)

hubbard_data = load_node(3075)
print(hubbard_data.get_ase())
print(hubbard_data.get_quantum_espresso_hubbard_card())

Atoms(symbols='Mn4O16P4Li4', pbc=True, cell=[[3.4378862545423e-06, 0.0, -4.740084535909], [0.0, -6.0917501186611, 0.0], [-10.474576328827, 0.0, -4.7800040115709e-06]], masses=...)
HUBBARD	ortho-atomic
 U	Mn-3d	4.5618
 V	Mn-3d	O-2p	1	518	0.0001
 V	Mn-3d	O-2p	1	601	0.0001
 V	Mn-3d	O-2p	1	373	0.0001
 V	Mn-3d	O-2p	1	621	0.0001
 V	Mn-3d	O-2p	1	326	0.0001
 V	Mn-3d	O-2p	1	17	0.0001
 V	Mn-3d	O-2p	1	596	0.0001
 V	Mn-3d	O-2p	2	15	0.0001
 V	Mn-3d	O-2p	2	324	0.0001
 V	Mn-3d	O-2p	2	12	0.0001
 V	Mn-3d	O-2p	2	8	0.0001
 V	Mn-3d	O-2p	2	19	0.0001
 V	Mn-3d	O-2p	2	328	0.0001
 V	Mn-3d	O-2p	2	5	0.0001
 V	Mn-3d	O-2p	3	18	0.0001
 V	Mn-3d	O-2p	3	17	0.0001
 V	Mn-3d	O-2p	3	11	0.0001
 V	Mn-3d	O-2p	3	371	0.0001
 V	Mn-3d	O-2p	3	378	0.0001
 V	Mn-3d	O-2p	3	377	0.0001
 V	Mn-3d	O-2p	3	6	0.0001
 V	Mn-3d	O-2p	4	20	0.0001
 V	Mn-3d	O-2p	4	19	0.0001
 V	Mn-3d	O-2p	4	626	0.0001
 V	Mn-3d	O-2p	4	6	0.0001
 V	Mn-3d	O-2p	4	632	0.0001
 V	Mn-3d	O-2p	4	631	0.0001
 V	Mn-3d	O-2p	4	7	0.0001



In [4]:
builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'olivine_iurii_overwrites.yaml')),
    spin_type=SpinType.COLLINEAR,
)

scf_dict = builder.scf.pw.parameters.get_dict()
relax_dict = builder.relax.base.pw.parameters.get_dict()
hubbard_dict = builder.hubbard.hp.parameters.get_dict()

print(json.dumps(scf_dict, sort_keys=False, indent=4))
print(json.dumps(relax_dict, sort_keys=False, indent=4))
print(json.dumps(hubbard_dict, sort_keys=False, indent=4))

{
    "CONTROL": {
        "calculation": "scf",
        "forc_conv_thr": 0.0001,
        "tprnfor": true,
        "tstress": true,
        "etot_conv_thr": 0.00028000000000000003
    },
    "SYSTEM": {
        "nosym": false,
        "occupations": "smearing",
        "smearing": "cold",
        "degauss": 0.01,
        "ecutwfc": 110,
        "ecutrho": 1320,
        "starting_magnetization": {
            "Mn": 0.3333333333333333,
            "O": 0.1,
            "P": 0.1,
            "Li": 0.1
        },
        "nspin": 2
    },
    "ELECTRONS": {
        "electron_maxstep": 300,
        "mixing_beta": 0.4,
        "conv_thr": 1e-10
    }
}
{
    "CONTROL": {
        "calculation": "vc-relax",
        "forc_conv_thr": 5e-05,
        "tprnfor": true,
        "tstress": true,
        "nstep": 200,
        "etot_conv_thr": 5e-06
    },
    "SYSTEM": {
        "nosym": false,
        "occupations": "smearing",
        "smearing": "cold",
        "degauss": 0.01,
        "ecutwfc": 11

In [6]:
# proper_olivine_builder = deepcopy(builder)
# proper_olivine_submit = submit(proper_olivine_builder)
# proper_olivine_submit_pk = proper_olivine_submit.pk
proper_olivine_submit_pk = 4282

In [7]:
!verdi process status 4282

SelfConsistentHubbardWorkChain<4282> Waiting [1:while_(should_run_iteration)(1:if_(should_run_relax))]
    ├── PwBaseWorkChain<4285> Finished [0] [4:results]
    │   ├── create_kpoints_from_distance<4286> Finished [0]
    │   └── PwCalculation<4290> Finished [0]
    ├── PwBaseWorkChain<4297> Finished [0] [4:results]
    │   ├── create_kpoints_from_distance<4298> Finished [0]
    │   └── PwCalculation<4302> Finished [0]
    ├── HpWorkChain<4308> Finished [0] [2:results]
    │   └── HpBaseWorkChain<4310> Finished [0] [3:results]
    │       └── HpCalculation<4312> Finished [0]
    ├── structure_relabel_kinds<4320> Finished [0]
    └── PwRelaxWorkChain<4324> Waiting [1:while_(should_run_relax)]
        └── PwBaseWorkChain<4327> Waiting [3:while_(should_run_process)(1:run_process)]
            ├── create_kpoints_from_distance<4328> Finished [0]
            └── PwCalculation<4332> Waiting
